# Imports

In [1]:
import os
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import random
import numpy as np
import string
import matplotlib.pyplot as plot
from scipy.stats import kstest,levene,ttest_rel, wilcoxon

#pytorch modules
import torch
from torch import load
from torchvision.utils import save_image
from torchvision import transforms
from torch.nn import MSELoss
from ignite.metrics import SSIM

#local modules
from configs import *
from utils import create_dirs_logs, compute_fid, compute_mse, save_list_table, draw_red_square, save_list_table
from data.common import unfold_image, fold_image, load_image_to_tensor, attention_mask_from_font

from models.diffusion_model import UNet

# Transform generated glyphs to complete fonts in a single image

In [3]:
source_dir="/home/francisco/logs/diffusion/default_conditional/test"
target_dir="/home/francisco/logs/diffusion/default_conditional/test_flat"
Path(target_dir).mkdir(exist_ok=True, parents=True)

fonts_list=os.listdir(os.path.join(source_dir,'0'))
for font_name in tqdm(fonts_list):
    font=torch.ones((1, 64, 64*26))
    for glyph_number in range(26):
        glyph=load_image_to_tensor(os.path.join(source_dir,str(glyph_number), font_name), unsqueeze=False, device='cpu')
        font[0, :, glyph_number*64:(glyph_number+1)*64] = glyph
    font=transforms.Lambda(lambda t: -t+1.)(font)
    save_image(font,os.path.join(target_dir,font_name))



# Random visuals

In [6]:
original_dir="/home/francisco/dataset/capitals64/test"
source_dir="/home/francisco/logs/diffusion/default_conditional/test_flat"
path_table="/home/francisco/logs/diffusion/default_conditional/data.txt"
data = []
meanMSE = 0
meanSSIM = 0

fonts_list=os.listdir(source_dir)
for font_name in tqdm(fonts_list):
    font_original=load_image_to_tensor(os.path.join(original_dir, font_name), unsqueeze=False, device='cuda:0')
    font_generated=load_image_to_tensor(os.path.join(source_dir, font_name), unsqueeze=False, device='cuda:0')

    mseLoss = MSELoss()
    mse = mseLoss(font_generated, font_original).tolist()
    meanMSE += mse

    ssimLoss = SSIM(data_range=1.0)
    ssimLoss.update((font_generated.unsqueeze(0), font_original.unsqueeze(0)))
    ssim = ssimLoss.compute()
    ssimLoss.reset()
    meanSSIM += ssim

    data.append([font_name[:-4], mse, ssim])

fid = compute_fid(original_dir, source_dir, batch_size=64, device='cuda:0')
data = [[meanMSE/len(fonts_list), meanSSIM/len(fonts_list), fid]]+data
save_list_table(path_table, data)

100%|██████████| 1560/1560 [01:02<00:00, 24.84it/s]
/home/francisco/miniconda3/envs/vae/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/francisco/miniconda3/envs/vae/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 88%|████████▊ | 22/25 [03:19<00:46, 15.36s/it]

100%|██████████| 25/25 [02:19<00:00,  5.58s/it]


In [9]:
target_folder='/home/francisco/dataset/capitals64/test'
generated_folder='/home/francisco/logs/experiment2/subset_2_seed_0/test'
second_generated_folder='/home/francisco/logs/diffusion/default_conditional/test_flat'

device="cuda:0"
input_glyphs=[7, 14, 3, 18, 6, 17]
fonts=[
    "47-Bold.0.0",
    "1980 portable.0.0",
    "Adieresis, Odieresis & Aring 2.0.0",
    "advanced_led_board-7.0.0",
    "Alpha54.0.0",
    #"AnagramNFShadow.0.0",
    "Antique Book Cover.0.0",
    "ARACNE_regular_italic.0.0",
    "BeautySchoolDropoutII.0.0",
    "betsy flanagan.0.0",
    "blackout_sunrise-webfont.0.0",
    "brassknuckle.0.0",
    "brick_led-7.0.0",
    "computer_7.0.0",
    #"Cruickshank.0.0",
    "gosebmps.0.0",
    #Walbaum-Fraktur.0.0",
    #"yonder recoil.0.0",
    ]

fonts=[font+'.png' for font in fonts]

image=torch.ones(size=(3,3,1664),device=device, requires_grad=False)

for name in tqdm(fonts):
    target=load_image_to_tensor(os.path.join(target_folder, name), unsqueeze=False, device=device)
    target=torch.cat((target,target,target),dim=0)

    for elem in input_glyphs:
        target=draw_red_square(target,startX=0,startY=64*elem,width=64)
    
    generated=load_image_to_tensor(os.path.join(second_generated_folder, name), unsqueeze=False, device=device)
    generated=torch.cat((generated,generated,generated),dim=0)

    parcel=torch.cat((target,generated),dim=1)

    generated=load_image_to_tensor(os.path.join(generated_folder, name), unsqueeze=False, device=device)
    generated=torch.cat((generated,generated,generated),dim=0)

    parcel=torch.cat((parcel,generated),dim=1)
    if name!=fonts[0]:
        image=torch.cat((image,torch.ones(size=(3,30,1664),device=device)),dim=1)
    else:
        image=torch.cat((image,torch.ones(size=(3,3,1664),device=device)),dim=1)
    image=torch.cat((image,parcel),dim=1)

save_image(image, f'/home/francisco/logs/diffusion/default_conditional/visuals.png')

100%|██████████| 14/14 [00:00<00:00, 80.92it/s]


# Simulate diffusion process

In [ ]:
with torch.no_grad():
    glyph_path="/home/francisco/diffusion-model/glyph.png"

    model = UNet(image_channels=1,n_channels=64,ch_mults=[1,2,3,4], n_blocks=2, device='cuda:0', max_beta=0.02)
    beta= torch.linspace(0.0001, 0.02, 1000)
    
    for timestep in [0,200,400,600,800,999]:
        print(beta[timestep])
        print(timestep)
        target=load_image_to_tensor(glyph_path,unsqueeze=True,device='cuda:0')
        target=transforms.Lambda(lambda t: -t+1.)(target)
        target=transforms.Lambda(lambda t: (t*2.-1.))(target)
        t = torch.tensor([timestep], device='cuda:0')
        noise = torch.randn_like(target)

        xt = model.q_sample(target, t, eps=noise)
        xt=transforms.Lambda(lambda t: -t+1.)(xt)
        
        save_image(xt,f'visual_{str(timestep)}.png')
        torch.cuda.empty_cache()